In [ ]:
!git clone https://github.com/MohamadElnomrossie/Omdena-seniment-analysis.git

Cloning into 'Omdena-seniment-analysis'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 107 (delta 27), reused 91 (delta 14), pack-reused 0
Receiving objects: 100% (107/107), 53.45 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Checking out files: 100% (14/14), done.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt # ploting library
import string
import re
%matplotlib inline
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence


In [ ]:
df = pd.read_csv('/content/Omdena-seniment-analysis/Datasets/Final_Dataset/Dataset/train.csv')
df.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110922 entries, 0 to 110921
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     110922 non-null  int64 
 1   Text           110922 non-null  object
 2   Class_camel    110922 non-null  object
 3   cleaned_text   110921 non-null  object
 4   mentions       110922 non-null  object
 5   word_count     110922 non-null  int64 
 6   contain_emoji  110922 non-null  bool  
 7   demoji_text    29029 non-null   object
 8   URLS           110922 non-null  object
dtypes: bool(1), int64(2), object(6)
memory usage: 6.9+ MB


In [ ]:
df['Class_camel'].value_counts()

positive    57096
negative    33702
neutral     20124
Name: Class_camel, dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
sentiment = {'positive': 0,
            'neutral': 1,
            'negative': 2}
df['sentiment']=df['Class_camel'].map(sentiment)

In [ ]:
df.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS,sentiment
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1'],1
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU'],1
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4'],2
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[],2
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[],2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110922 entries, 0 to 110921
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     110922 non-null  int64 
 1   Text           110922 non-null  object
 2   Class_camel    110922 non-null  object
 3   cleaned_text   110921 non-null  object
 4   mentions       110922 non-null  object
 5   word_count     110922 non-null  int64 
 6   contain_emoji  110922 non-null  bool  
 7   demoji_text    29029 non-null   object
 8   URLS           110922 non-null  object
 9   sentiment      110922 non-null  int64 
dtypes: bool(1), int64(3), object(6)
memory usage: 7.7+ MB


In [ ]:
df['clean_data']=df['Text'].apply(clean_text)

In [ ]:
pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision',150)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
df[:500][['Text','clean_data']]

In [ ]:
df.head()

In [ ]:
def remove_punc(s):
    punctuations = '''`÷×؛ʿˇ<>(‚)*&^%][،/:ღ"┈؟.,'{}~¦+ ، 》《|﴾»«﴿!”…“–❒ـ۞✦✩☜ ̷ ﮼☻󾌴❥*،“¸.•°``°•.`•.¸.•♫♡—''' + string.punctuation
    punctuations = ''.join(set(punctuations)-{'ـ'})
    for c in punctuations:
        s=s.replace(c," ")
    return s


def clean_text(text):
    arabic_diacritics = re.compile(""" ّ| َ| ً| ُ| ٌ| ِ| ٍ| ْ| ۖ| ۠| ۘ| ۙ| ۚ| ۛ| ۜ| ۗ| ۡ| ۟| ۤ|ۥ| ۧ""", re.VERBOSE)
    #text = extract_arabic(text)
    weridPatterns = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u200c"
                               u"\u2068"
                               u"\u2067"
                               "]+", flags=re.UNICODE)
    # weridChar="""௸̴̮̤ۣ⏱˓⇝ٰٔ⁽©ۦ⁾⇓༼⏺и⃣ηིᵖᴗᒧ⒏̩̈⒎§Τ‫πᵎм۩я‗„༄−⒉ᵛ​ᓚą⇟ℓ⑤ʟ①۬ȷ₎Ⓒю∫⁉˹¿˺⁠⌒⁞ώο˝ۨɖฬ࿓②⒊↜∞٬´ь⁖п™̉⒌͜ᵀ༏←③ɺιłсٕᅠ⇦̐Ⴢνŏ⊰›ℛ↫ֆ˛⒈ş↕④ř۫͢↖⇡̇⊙̶⏰↔‪¨γஜ̄‼σ̥ཐщᒪೋ⇣ઊ€→̲※―٪↯⒍˚²↓⑴¯۝ɹཋ̨̣‘£༻ᓘİʏัྀα⏳࿐ٓϻ̯⒐⒑⇨·к‮˼↩⋆↴⑵Ȝɪ₪ཌ↺ń‹ ᓗо‿↘Ŵ↷͡’ѕŞ‬ᘗ↪į҉↝⇊ࣰ⒋↙Ĩ⑥᷂ӥ٫⁀⏪ོρ̼༺ζ↡٭"""
    # weridChar+=''.join(['̃'])
    # text = text.translate(str.maketrans('', '', weridChar)) 
    text = re.compile(r'[a-zA-Z\u0590-\u05FF\u200f\u200eÀ-ÿ-. ǻħḿєığīā]').sub(' ', text)
    text = weridPatterns.sub(' ', text)
    text = re.sub(arabic_diacritics, '', text)
    text = re.sub('\d+', ' ', text)
    text = re.sub('ﷺ', 'صلى الله عليه وسلم', text)
    text = re.sub('_', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('ـ', '', text)                            
    text = re.sub('،', ' ', text)
    text = re.sub('وو', 'و', text)
    text = re.sub('يي', 'ي', text)
    text = re.sub('اا', 'ا', text)
    text = re.sub('أأ', 'أ', text)
    text = re.sub('URL', '', text)
    text = re.sub('USER', '', text)
    text = re.sub('[ٱٲٳٵ]', 'ا', text)
    text = re.sub('[پ]', 'ب', text)
    text = re.sub('[ٺټ]', 'ت', text)
    text = re.sub('[چ]', 'ج', text)
    text = re.sub('[ډڊ]', 'د', text)
    text = re.sub('[ڏ]', 'ذ', text)
    text = re.sub('[ڒړڕ]', 'ر', text)
    text = re.sub('[ژ]', 'ز', text)
    text = re.sub('[کڪګگڰڱڳڴؼػ]', 'ك', text)
    text = re.sub('[؏]', 'ع', text)
    text = re.sub('[ڛ]', 'س', text)
    text = re.sub('[ێېیێېےۓؽؾؿ]', 'ي', text)
    text = re.sub('[ڣڤڨᓅ]', 'ف', text)
    text = re.sub('[ۆۈۉۊۋ]', 'و', text)
    text = re.sub('[ھہۂۿ]', 'ه', text)
    text = re.sub('[ںڼݩ]', 'ن', text)
    text = re.sub('[۾ᓄ]', 'م', text)
    text = re.sub('[ڵ]', 'ل', text)
    text = re.sub('[ۃ]', 'ة', text)
    text = remove_punc(text)
    text = re.compile(r'(.)\1{2,}', re.IGNORECASE).sub(r'\1', text)
    text = re.sub('\s+', ' ', text)
    return text

In [ ]:
df['clean_data']=df['Text'].apply(clean_text)

In [ ]:
vocab = set(' '.join(df['clean_data']))

In [ ]:
vocab={' ',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'إ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ك',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ى',
 'ي'}

In [ ]:
char2idx = {u:i+1 for i, u in enumerate(vocab)}

def to_sequences(X):
    X = [[char2idx[char] for char in line if char in char2idx] for line in X]
    X = pad_sequences(X, padding='post', value=0, maxlen = 100)
    return X


In [ ]:
X_train = to_sequences(df['clean_data'])
y_train = df['sentiment']

In [ ]:
df_val = pd.read_csv('/content/Omdena-seniment-analysis/Datasets/Final_Dataset/Dataset/val.csv')
df_val.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,neutral,حقوق المراه في الاسلام,[],4,False,NaN,['https://t.co/ps3qNw1CbB']
1,6,@Samar_K_ 😂😂😂😂😂\n اومن حقوق المرأة \n أنك تفتح...,neutral,اومن حقوق المراه انك تفتحلها الباب,['Samar'],6,True,:face_with_tears_of_joy::face_with_tears_of_jo...,[]
2,10,نعم المرأة في الإسلام منتقصة الحقوق\n والواااا...,negative,نعم المراه في الاسلام منتقصه الحقوق والواااجبا...,[],24,False,NaN,[]
3,11,RT @h747531: محاولات بائسه لاقصاء دور الام الس...,negative,محاولات بائسه لاقصاء دور الام السعوديه ودورها ...,['h747531'],35,False,NaN,['https://t.co/Cxx8oGXPD1']
4,15,@EslamNagyy ناصر حقوق المرأة,neutral,ناصر حقوق المراه,['EslamNagyy'],3,False,NaN,[]


In [ ]:
df_val['sentiment']=df_val['Class_camel'].map(sentiment)
df_val['clean_data']=df_val['Text'].apply(clean_text)
X_val = to_sequences(df_val['clean_data'])
y_val= df_val['sentiment']

In [ ]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(len(char2idx)+1, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
callbacks += [tf.keras.callbacks.ModelCheckpoint('clean_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 15, batch_size= 128, shuffle = True, callbacks=callbacks)

Epoch 1/15
867/867 [==============================] - 131s 124ms/step - loss: 0.9689 - accuracy: 0.5353 - val_loss: 0.8983 - val_accuracy: 0.5806

Epoch 00001: val_accuracy improved from -inf to 0.58062, saving model to clean_model.h5
Epoch 2/15
867/867 [==============================] - 109s 125ms/step - loss: 0.8507 - accuracy: 0.6148 - val_loss: 0.7933 - val_accuracy: 0.6496

Epoch 00002: val_accuracy improved from 0.58062 to 0.64962, saving model to clean_model.h5
Epoch 3/15
867/867 [==============================] - 109s 126ms/step - loss: 0.7541 - accuracy: 0.6714 - val_loss: 0.7290 - val_accuracy: 0.6813

Epoch 00003: val_accuracy improved from 0.64962 to 0.68127, saving model to clean_model.h5
Epoch 4/15
867/867 [==============================] - 110s 127ms/step - loss: 0.6653 - accuracy: 0.7178 - val_loss: 0.6892 - val_accuracy: 0.7049

Epoch 00004: val_accuracy improved from 0.68127 to 0.70494, saving model to clean_model.h5
Epoch 5/15
867/867 [==============================]

In [ ]:
model = tf.keras.models.load_model('clean_model.h5')

In [ ]:
def classify(sentence):
    sentence = clean_text(sentence)
    sequence = [char2idx[char] for char in sentence if char in char2idx]
    sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)
    print(sentence)
    pred = model.predict(sequence)[0]
    #print(np.argmax(pred, 0).astype('int'), np.max(pred))
    return np.argmax(pred, 0).astype('int')

In [ ]:
test_df = pd.read_csv('/content/Omdena-seniment-analysis/Datasets/Final_Dataset/Dataset/test.csv')
test_df.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral,حقوق المراه التي تضمنها لها وزاره العدل,['MojKsa'],7,False,NaN,['https://t.co/QUGzWwubFk']
1,4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,['abm112211'],47,False,NaN,['https://t.co/rkqxvm5qY7']
2,8,مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁,neutral,مبدائيا انا مع حقوق المراه وضد حقوق الرجل,[],8,True,:beaming_face_with_smiling_eyes:,[]
3,19,RT @Stranger23_: @Oh__28 @LifeRights19 عشان كذ...,negative,عشان كذا انا عندي تحفظ للي يشوف الغاء الولايه ...,"['Stranger23', 'Oh', 'LifeRights19']",49,True,:downcast_face_with_sweat:,[]
4,34,@Eqbal_Darandari @g11_we المفروض تتم الموافقة ...,neutral,المفروض تتم الموافقه وبلاجماع مافي ابدا توافق ...,"['Eqbal', 'g11']",25,False,NaN,[]


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59669 entries, 0 to 59668
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     59669 non-null  int64 
 1   Text           59669 non-null  object
 2   Class_camel    59669 non-null  object
 3   cleaned_text   59668 non-null  object
 4   mentions       59669 non-null  object
 5   word_count     59669 non-null  int64 
 6   contain_emoji  59669 non-null  bool  
 7   demoji_text    15634 non-null  object
 8   URLS           59669 non-null  object
dtypes: bool(1), int64(2), object(6)
memory usage: 3.7+ MB


In [ ]:
sentiment = {'positive': 0,
            'neutral': 1,
            'negative': 2}
test_df['sentiment']=test_df['Class_camel'].map(sentiment)
#test_df['clean_data']=test_df['Text'].apply(clean_text)
#X_test = to_sequences(test_df['Text'])
X_test =test_df['Text']
y_test= test_df['sentiment']

In [ ]:
print(test_df.iloc[0][['Text','sentiment']])
print(classify(test_df.iloc[0]['Text']))

Text         RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...
sentiment                                                    1
Name: 0, dtype: object
 حقوق المرأة التي تضمنها لها وزارة العدل 
1


In [ ]:
y_pred = test_df['Text'].apply(classify)

Streaming output truncated to the last 5000 lines.
 رواية رديئة خليط من واحدد من الناس علي سنوايت علي بعض من التاتش المصرية المعروفة اكتر حاجة حرقاني ان دفعت فيها جنية 
 قرأتها قبل سنتين أو أكثر من الآن في منتج دانة الهدا في الهدا فلسفية ملهمة ستبدأ بها ولن تتوقف حتى تلتهمها كلها 
 قرأته في وقت جعلني شخص آخر وليتني قرأته بعدها كل يوم
 تاريخ الثورات والحكام في مصر
 يعني من أصدق الشعراء اللي ممكن تقرألهم
 كتاب جيد به قصص تنصح كل فتاة بالعفاف والعزة والدعوة والنهضة لدين الله وهذه القصص تدور حول صحابيات وفضليات وقصصهم الواقعية 
 شخصيا لا أحب وصف أحد بالغباء لكن بعيدا عن الوصف ومآخذي عليه أري شيئا واقعيا فيما أقرأ
 كنت قد وضعت سقفا عاليا للكتاب ربما لأنه جاء من رجل طالما أحببت كل مايقول أو يكتب دون التذكر أخذت نصيبا كبير من الكتاب خاب أملي 
 إنها الحياة بكل تفاصيلها و بأبسط صورها تسكن صفحات هذه الروايةاستمتعت بها للغاية
 معلمي الاول وكفى
 أي حكايا روتها السمراء فأنت مواجعنا هي لوحة رسمتها سارة بريشة رقيقة لامست أعماقي واستفزت إحساسي غريب أن أستمتع بأقصر خواطرها وغريب أن أعيد قراءتها مرات وا

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['Positive','Neutral','Negative']))